In [1]:
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import numpy as np
import os

Face Pattern Recognition

In [12]:
#Define image to work with. It represents the real classroom photo taken with the camera
classroomFolder=r"C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos"
#Select the model to be used for detection varying the list index
detectors = [ "mtcnn","opencv", "retinaface"]
#Change depending on the index to be evaluates
chosen_detector_index=2

m=1
#Iterate through all real classroom photos
for file in os.listdir(classroomFolder):
    facesCoordinatesList=[]
    camera_local_file = rf'C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos\{file}' 

    image = cv2.imread(camera_local_file)
    faces = DeepFace.extract_faces(img_path=camera_local_file, detector_backend=detectors[2])
    #Obtain images dimensions
    image_height, image_width, _ = image.shape

    #Save the coordinates of all faces in order to paint it later
    for i in range(len(faces)):
        facesCoordinatesList.append(faces[i]["facial_area"])
    # Draw bounding boxes for each detected face using the coordinates
    for element in facesCoordinatesList:
        left = element["x"]
        top = element["y"]
        width = element["w"]
        height =  element["h"]
        cv2.rectangle(
            image, 
            (left, top), 
            (left + width, top + height), 
            (0,0,200), 
            2  
        )

    # Print number of faces detected
    print(f"{len(facesCoordinatesList)} Faces were detected in {camera_local_file}")

    # Save the annotated image in the destinated folder
    output_path = rf"C:\Users\User\Desktop\FinalProject\ImagesDeepface\Detection\Detection{detectors[chosen_detector_index]}\faces_detected_{m}.jpeg"
    
    cv2.imwrite(output_path, image)
    #Increase the photo's index
    m=m+1


20 Faces were detected in C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos\classroom1.jpeg
14 Faces were detected in C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos\classroom2.jpeg


Face Recognition

In [ ]:
student_names_array=["AlejandroCallasas", "AndresAnillo","DiegoGomez","GabrielCastanez", "MauricioDeLaHoz","YordiRochel", "MariaBerdugo","AlbaQuiroz","GiohanOlivares","JesusCotes"]
colors=[(0, 200, 0),(200,0,0),(0,0,0), (0,0,255), (150,150,0),(0,150,150),(100,20,60),(20,60,100),(60,100,20),(255,255,255)]
recognition_models=["ArcFace","Facenet", "VGG-Face"]
chosen_model_index=1

classroomFolder=r"C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos"
model=recognition_models[chosen_model_index]


#os.makedirs(output_folder, exist_ok=True)
m=1
for file in os.listdir(classroomFolder):
    camera_local_file=rf"C:\Users\User\Desktop\FinalProject\ImagesDeepface\RealClassroomPhotos\{file}"
    #The image read of the classroom. This will be used to redraw it
    camera_photo = cv2.imread(camera_local_file)
    recognized_counter=0
    k=0
    for student_name in student_names_array:
        student_photo_path=rf"C:\Users\User\Desktop\FinalProject\ImagesDeepface\Students\{student_name}.jpeg"

        result = DeepFace.verify(
                    img1_path=student_photo_path, #individual
                    img2_path=camera_local_file,
                    model_name=model,
                    detector_backend="mtcnn",
                    enforce_detection=False
                    )
        coordinatesOfGroupPhoto=result["facial_areas"]["img2"]
        x=coordinatesOfGroupPhoto["x"]
        y=coordinatesOfGroupPhoto["y"]
        h=coordinatesOfGroupPhoto["h"]
        w=coordinatesOfGroupPhoto["w"]


            # Cargar la imagen
        if camera_photo is None:
            raise FileNotFoundError(f"No se pudo cargar la imagen: {classroomFolder}")

        # Dibujar el rectángulo
        if str(result["verified"])=="True":
            x, y, w, h = coordinatesOfGroupPhoto["x"], coordinatesOfGroupPhoto["y"], coordinatesOfGroupPhoto["w"], coordinatesOfGroupPhoto["h"]
            cv2.rectangle(camera_photo, (x, y), (x + w, y + h), colors[k], 2)  # Color verde, grosor 2
            padding = 5  # Espacio entre el rectángulo y el cuadro de texto
            text_box_height = 25  # Altura del cuadro de texto
            # Agregar texto dentro del recuadro de texto
            cv2.putText(camera_photo, f"{student_name}", (left + 10, top - padding - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[k], 1)
            print(f"{student_name} was recognized in photo {m}")
            recognized_counter=recognized_counter+1
        else:
            print(f"{student_name} was NOT recognized in photo {m}")

        k=k+1

    output_path = rf"C:\Users\User\Desktop\FinalProject\ImagesDeepface\Recognition\Recognition{model}\faces_in_classroom{m}_{recognized_counter}.jpg"
        # Guardar la imagen modificada
    cv2.imwrite(output_path, camera_photo)
    print(f"Imagen guardada en: {output_path}")
    m=m+1

Mathematical metrics for analizing facial similarity

In [4]:
from scipy.spatial.distance import euclidean,cosine

#Obtain the analysis of the image
embedding1=DeepFace.represent(img_path="single2.jpeg",model_name="ArcFace", detector_backend="retinaface")
embedding2=DeepFace.represent(img_path="single11.jpeg",model_name="ArcFace", detector_backend="retinaface")

#Extract the multi-dimensional vector only
embedding1Vector1=np.array(embedding1[0]["embedding"])
embedding1Vector2=embedding2[0]["embedding"]

def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

print("Cosine similarity is "+ str(cosine_similarity(embedding1Vector1,embedding1Vector2)))

def euclidean_distance(embedding1,embedding2):
    return euclidean(embedding1,embedding2)

print("Euclidean distance is "+ str(euclidean_distance(embedding1Vector1,embedding1Vector2)))



Cosine similarity is 0.35477153678865364
Euclidean distance is 3.8419692767006404
